In [ ]:
using LatticePhysics
using LatPhysBandstructures
using LatPhysPlottingPyPlot
using LatPhysBandstructuresPlottingPyPlot

using LinearAlgebra
using Distributed
using BenchmarkTools

import LatPhysBandstructures.hamiltonian
import LatPhysBandstructures.kpoints
import LatPhysBandstructures.energies
import LatPhysBandstructures.recalculate!
import LatPhysBandstructures.numKPoints
import LatPhysBandstructures.path

In [ ]:
uc = getUnitcellHyperhoneycomb()
h  = Hamiltonian(uc, getHoppingHamiltonianSimple(uc))
bs = getBandstructure(h, :Gamma, :K, :M, :Gamma);

In [ ]:
@btime recalculate!(bs)

In [ ]:
plotBandstructure(bs);

# K Point sampling

In [ ]:
function getKPointsGrid(
        bz :: BZ,
        N_points :: Integer
    ) where {D,L,P<:AbstractReciprocalPoint{D},B<:AbstractBond{L,2},RU<:AbstractReciprocalUnitcell{P,B},BZ<:AbstractBrillouinZone{RU}}

    # obtain linear dimension
    lin_N = round(Int64, sqrt(N_points))
    # obtain reciprocal unitcell
    ruc = reciprocalUnitcell(bz)
    # build list of points that where shifted to first BZ
    kpoints = pmap(
        ab -> shiftToFirstBZ(ruc, ab[1]*a1(ruc) .+ ab[2]*a2(ruc)),
        [(a,b) for a in range(0.0,stop=1.0,length=lin_N+1)[1:end-1] for b in range(0.0,stop=1.0,length=lin_N+1)[1:end-1]]
    )
    # return the points
    return kpoints
end
function getKPointsGrid(
        bz :: BZ,
        N_points :: Integer
    ) where {D,L,P<:AbstractReciprocalPoint{D},B<:AbstractBond{L,3},RU<:AbstractReciprocalUnitcell{P,B},BZ<:AbstractBrillouinZone{RU}}

    # obtain linear dimension
    lin_N = round(Int64, (N_points)^(1/3))
    # obtain reciprocal unitcell
    ruc = reciprocalUnitcell(bz)
    # build list of points that where shifted to first BZ
    kpoints = pmap(
        ab -> shiftToFirstBZ(ruc, ab[1]*a1(ruc) .+ ab[2]*a2(ruc) .+ ab[3]*a3(ruc)),
        [(a,b,c) for a in range(0.0,stop=1.0,length=lin_N+1)[1:end-1] for b in range(0.0,stop=1.0,length=lin_N+1)[1:end-1] for c in range(0.0,stop=1.0,length=lin_N+1)[1:end-1]]
    )
    # return the points
    return kpoints
end

function getKPointsRandom(
        bz :: BZ,
        N_points :: Integer
    ) where {D,L,P<:AbstractReciprocalPoint{D},B<:AbstractBond{L,2},RU<:AbstractReciprocalUnitcell{P,B},BZ<:AbstractBrillouinZone{RU}}

    # obtain reciprocal unitcell
    ruc = reciprocalUnitcell(bz)
    # build list of points that where shifted to first BZ
    kpoints = pmap(
        i -> shiftToFirstBZ(ruc, rand()*a1(ruc) .+ rand()*a2(ruc)),
        1:N_points,
        batch_size=100
    )
    # return the points
    return kpoints
end
function getKPointsRandom(
        bz :: BZ,
        N_points :: Integer
    ) where {D,L,P<:AbstractReciprocalPoint{D},B<:AbstractBond{L,3},RU<:AbstractReciprocalUnitcell{P,B},BZ<:AbstractBrillouinZone{RU}}

    # obtain reciprocal unitcell
    ruc = reciprocalUnitcell(bz)
    # build list of points that where shifted to first BZ
    kpoints = pmap(
        i -> shiftToFirstBZ(ruc, rand()*a1(ruc) .+ rand()*a2(ruc) .+ rand()*a3(ruc)),
        1:N_points,
        batch_size=100
    )
    # return the points
    return kpoints
end

# Abstract ED Result & concrete type

In [ ]:
abstract type AbstractEDResult{H<:AbstractHamiltonian{L,UC,HB} where {L,UC,HB}} end

In [ ]:
function hamiltonian(
        aedr :: AEDR
    ) :: H where {H, AEDR <: AbstractEDResult{H}}
    
    # print an error because implementation for concrete type is missing
    error("not implemented interface function 'hamiltonian' for abstract ED result of type " * string(AEDR))
end

function kpoints(
        aedr :: AEDR
    ) where {H, AEDR <: AbstractEDResult{H}}
    
    # print an error because implementation for concrete type is missing
    error("not implemented interface function 'kpoints' for abstract ED result of type " * string(AEDR))
end

function energies(
        aedr :: AEDR
    ) where {H, AEDR <: AbstractEDResult{H}}
    
    # print an error because implementation for concrete type is missing
    error("not implemented interface function 'energies' for abstract ED result of type " * string(AEDR))
end




function recalculate!(
        aedr :: AEDR
        ;
        kwargs...
    ) where {H, AEDR <: AbstractEDResult{H}}
    
    # print an error because implementation for concrete type is missing
    error("not implemented interface function 'recalculate!' for abstract ED result of type " * string(AEDR))
end




function numKPoints(
        aedr :: AEDR
    ) where {H, AEDR <: AbstractEDResult{H}}
    
    # return length of k points list
    return length(kpoints(aedr))
end

In [ ]:
mutable struct EDResult{H} <: AbstractEDResult{H}
    
    # Hamiltonian
    h :: H
    
    # list of k points
    k_vals :: Vector{Vector{Float64}}
    
    # list of energy values
    e_vals :: Vector{Vector{Float64}}
    
end


function EDResult(
        h :: H
    ) :: EDResult{H} where {H}
    
    return EDResult{H}(h, Vector{Float64}[], Vector{Float64}[])
end

In [ ]:
function hamiltonian(
        edr :: EDResult{H}
    ) :: H where {H}
    
    # return the hamiltonian
    return edr.h
end

function kpoints(
        edr :: EDResult{H}
    ) where {H}
    
    # return k values
    return edr.k_vals
end

function energies(
        edr :: EDResult{H}
    ) where {H}
    
    # return energy values
    return edr.e_vals
end



function recalculate!(
        edr :: EDResult{H}
        ;
        N :: Integer = -1,
        mesh_type :: Symbol = :grid,
        kwargs...
    ) where {H}
    
    # obtain hamiltonian
    h = hamiltonian(edr)
    
    # check if new k points wanted
    if N > 0
        # check how to sample
        if mesh_type == :grid || mesh_type == :Grid
            # create a grid of k points
            edr.k_vals = getKPointsGrid(getBrillouinZone(unitcell(h)), N)
        elseif mesh_type == :rand || mesh_type == :random || mesh_type == :Rand || mesh_type == :Random
            # create a grid of k points
            edr.k_vals = getKPointsRandom(getBrillouinZone(unitcell(h)), N)
        else
            error("Want $(N) new K points but gave unreadable mesh type :$(mesh_type)")
        end
    end
    
    # diagonalize for every k point again
    edr.e_vals = pmap(
        k -> eigvals!(Hermitian(matrixAtK(h, k))),
        kpoints(edr)
    )
    
    # return nothing
    return nothing
end

In [ ]:
edr = EDResult(h)
edr.k_vals = [rand(2) for i in 1:300];
recalculate!(edr, N=100, mesh_type=:grid)

In [ ]:
@btime recalculate!(edr, N=300, mesh_type=:random)

# Abstract Linear ED Result & concrete type

In [ ]:
abstract type AbstractLinearEDResult{H} <: AbstractEDResult{H} end

In [ ]:
function hamiltonian(
        aledr :: ALEDR
    ) :: H where {H, ALEDR <: AbstractLinearEDResult{H}}
    
    # print an error because implementation for concrete type is missing
    error("not implemented interface function 'hamiltonian' for abstract linear ED result of type " * string(ALEDR))
end

function alphas(
        aledr :: ALEDR
    ) where {H, ALEDR <: AbstractLinearEDResult{H}}
    
    # print an error because implementation for concrete type is missing
    error("not implemented interface function 'alphas' for abstract linear ED result of type " * string(ALEDR))
end

function k1(
        aledr :: ALEDR
    ) where {H, ALEDR <: AbstractLinearEDResult{H}}
    
    # print an error because implementation for concrete type is missing
    error("not implemented interface function 'k1' for abstract linear ED result of type " * string(ALEDR))
end

function k2(
        aledr :: ALEDR
    ) where {H, ALEDR <: AbstractLinearEDResult{H}}
    
    # print an error because implementation for concrete type is missing
    error("not implemented interface function 'k2' for abstract linear ED result of type " * string(ALEDR))
end

function energies(
        aledr :: ALEDR
    ) where {H, ALEDR <: AbstractLinearEDResult{H}}
    
    # print an error because implementation for concrete type is missing
    error("not implemented interface function 'energies' for abstract linear ED result of type " * string(ALEDR))
end





function kpoints(
        aledr :: ALEDR
    ) where {H, ALEDR <: AbstractLinearEDResult{H}}
    
    # obtain values
    alpha_vals = alphas(aledr)
    k1_val     = k1(aledr)
    k2_val     = k2(aledr)
    
    # build array of k values
    return Vector{Float64}[
        ((k1_val .* (1-a)) .+ (k2_val .* a))
        for a in alpha_vals
    ]
end

function numKPoints(
        aledr :: ALEDR
    ) where {H, ALEDR <: AbstractLinearEDResult{H}}
    
    # return length of k points list
    return length(alphas(aledr))
end




function recalculate!(
        aledr :: ALEDR
        ;
        kwargs...
    ) where {H, ALEDR <: AbstractLinearEDResult{H}}
    
    # print an error because implementation for concrete type is missing
    error("not implemented interface function 'recalculate!' for abstract linear ED result of type " * string(ALEDR))
end

In [ ]:
mutable struct LinearEDResult{H} <: AbstractLinearEDResult{H}
    
    # Hamiltonian
    h :: H
    
    # k1 and k2 (start and end of linear segment)
    k1 :: Vector{Float64}
    k2 :: Vector{Float64}
    
    # list of alpha values
    a_vals :: Vector{Float64}
    
    # list of energy values
    e_vals :: Vector{Vector{Float64}}
    
end


function LinearEDResult(
        h :: H
    ) :: LinearEDResult{H} where {H}
    
    return LinearEDResult{H}(h, point(site(unitcell(h),1)).*0.0, point(site(unitcell(h),1)).*0.0, Float64[], Vector{Float64}[])
end

function LinearEDResult(
        h  :: H,
        k1 :: Vector{<:Real},
        k2 :: Vector{<:Real}
    ) :: LinearEDResult{H} where {H}
    
    return LinearEDResult{H}(h, k1,k2, Float64[], Vector{Float64}[])
end

In [ ]:
function hamiltonian(
        ledr :: LinearEDResult{H}
    ) :: H where {H}
    
    # return the hamiltonian
    return ledr.h
end

function alphas(
        ledr :: LinearEDResult{H}
    ) where {H}
    
    # return alpha values
    return ledr.a_vals
end

function k1(
        ledr :: LinearEDResult{H}
    ) where {H}
    
    # return k1 value
    return ledr.k1
end
function k2(
        ledr :: LinearEDResult{H}
    ) where {H}
    
    # return k2 value
    return ledr.k2
end

function energies(
        ledr :: LinearEDResult{H}
    ) where {H}
    
    # return energy values
    return ledr.e_vals
end



function recalculate!(
        ledr :: LinearEDResult{H}
        ;
        N :: Integer = -1,
        mesh_type :: Symbol = :grid,
        kwargs...
    ) where {H}
    
    # obtain hamiltonian
    h = hamiltonian(ledr)
    
    
    # check if new k points wanted
    if N > 0
        # check how to sample
        if mesh_type == :grid || mesh_type == :Grid
            # create a grid of k points
            ledr.a_vals = collect(range(0, stop=1, length=N))
        elseif mesh_type == :rand || mesh_type == :random || mesh_type == :Rand || mesh_type == :Random
            # create a grid of k points
            ledr.a_vals = sort(rand(N))
        else
            error("Want $(N) new K points but gave unreadable mesh type :$(mesh_type)")
        end
    end
    
    # diagonalize for every k point again
    ledr.e_vals = pmap(
        k -> eigvals!(Hermitian(matrixAtK(h, k))),
        kpoints(ledr)
    )
    
    # return nothing
    return nothing
end

In [ ]:
ledr = LinearEDResult(h, point(path(bs)[1]), point(path(bs)[2]))
ledr.a_vals = sort(rand(300))
recalculate!(ledr, N=100)

In [ ]:
@btime recalculate!(ledr, N=300, mesh_type=:grid)

# Bandstructures

In [ ]:
abstract type AbstractNewBandstructure{
    P<:AbstractReciprocalPath{RP} where {RP},
    H<:AbstractHamiltonian{L,UC,HB} where {L,UC,HB},
    ALEDR<:AbstractLinearEDResult{H}
} <: AbstractEDResult{H} end

In [ ]:
function hamiltonian(
        bs :: AB
    ) :: H where {H, P, ALEDR, AB <: AbstractNewBandstructure{P,H,ALEDR}}
    
    # print an error because implementation for concrete type is missing
    error("not implemented interface function 'hamiltonian' for abstract bandstructure of type " * string(AB))
end

function segments(
        bs :: AB
    ) :: Vector{ALEDR} where {H, P, ALEDR, AB <: AbstractNewBandstructure{P,H,ALEDR}}
    
    # print an error because implementation for concrete type is missing
    error("not implemented interface function 'segments' for abstract bandstructure of type " * string(AB))
end

function path(
        bs :: AB
    ) :: P where {H, P, ALEDR, AB <: AbstractNewBandstructure{P,H,ALEDR}}
    
    # print an error because implementation for concrete type is missing
    error("not implemented interface function 'path' for abstract bandstructure of type " * string(AB))
end




function recalculate!(
        bs :: AB
        ;
        N :: Integer = -1,
        mesh_type :: Symbol = :grid,
        kwargs...
    ) where {H, P, ALEDR, AB <: AbstractNewBandstructure{P,H,ALEDR}}
    
    # recalculate every segment
    recalculate!.(segments(bs); N=round(Int64, N./length(segments(bs))), mesh_type=mesh_type, kwargs...)
    
    # return nothing
    return nothing
end

function kpoints(
        bs :: AB
    ) where {H, P, ALEDR, AB <: AbstractNewBandstructure{P,H,ALEDR}}
    
    # vcat the segment data
    return vcat(kpoints.(segments(bs)) ...)
end

function energies(
        bs :: AB
    ) where {H, P, ALEDR, AB <: AbstractNewBandstructure{P,H,ALEDR}}
    
    # vcat the segment data
    return vcat(energies.(segments(bs)) ...)
end

function numKPoints(
        aedr :: AEDR
    ) where {H, AEDR <: AbstractEDResult{H}}
    
    # return length of k points list
    return sum(numKPoints.(segments(bs)))
end

In [ ]:
mutable struct NewBandstructure{P,H,S} <: AbstractNewBandstructure{P,H,S}
    
    # Hamiltonian
    h :: H
    
    # Reciprocal Path
    path :: P
    
    # segments
    segments :: Vector{S}
    
end


function NewBandstructure(
        h :: H,
        p :: P
    ) :: NewBandstructure{P,H,LinearEDResult{H}} where {H,P}
    
    return NewBandstructure{P,H,LinearEDResult{H}}(
        h, 
        p,
        LinearEDResult{H}[LinearEDResult(h, point(p[i]), point(p[i+1])) for i in 1:length(p)-1]
    )
end

In [ ]:
function hamiltonian(
        bs :: NewBandstructure{P,H,ALEDR}
    ) :: H where {H, P, ALEDR}
    
    # return the field
    return bs.h
end

function segments(
        bs :: NewBandstructure{P,H,ALEDR}
    ) :: Vector{ALEDR} where {H, P, ALEDR}
    
    # return the field
    return bs.segments
end

function path(
        bs :: NewBandstructure{P,H,ALEDR}
    ) :: P where {H, P, ALEDR}
    
    # return the field
    return bs.path
end

In [ ]:
bs = NewBandstructure(h, path(bs));
recalculate!(bs, N=300)

In [ ]:
@btime recalculate!(bs, N=300, mesh_type=:rand)

In [ ]:
@btime recalculate!(bs, N=300, mesh_type=:grid)
@btime recalculate!(bs, N=300, mesh_type=:rand)
@btime recalculate!(bs)